In [1]:
import numpy as np
grasp_data = np.load("./contact_points_merged.npy")

In [2]:
grasp_data.shape

(590, 3)

In [3]:
import open3d as o3d
import numpy as np

import gpytorch

import time
import torch
import itertools
import plotly.graph_objects as go

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
res = 100 # 150 # grid resolution # 50
k_nearest=5
display_percentile_low=10
display_percentile_high=90
training_iter = 200
grid_count = 10

In [5]:
temp_min = np.min(grasp_data, axis=0)
temp_max = np.max(grasp_data, axis=0)
fone   = np.ones((grasp_data.shape[0],)) 
min_data = temp_min - (temp_max-temp_min)*0.2 # We extend the boundaries of the object a bit to evaluate a little bit further 
max_data = temp_max + (temp_max-temp_min)*0.2


x_axis = np.linspace(min_data[0], max_data[0],grid_count)
y_axis = np.linspace(min_data[1], max_data[1],grid_count)
z_axis = np.linspace(min_data[2], max_data[2],grid_count)

In [6]:
train_X,train_y = [], []
for x in x_axis:
    for y in y_axis:
        for z in z_axis:
            train_X.append([x,y,z])
            train_y.append(0)

train_X = np.array(train_X)
train_y = np.array(train_y)

In [8]:
train_X.shape

(1000, 3)

In [10]:
train_y.shape

(1000,)

In [11]:
gridpcd = o3d.geometry.PointCloud()
gridpcd.points = o3d.utility.Vector3dVector(train_X)

kdtree_gridpcd = o3d.geometry.KDTreeFlann(gridpcd)

In [12]:
index = []

for p in grasp_data:
    [k, idx, dis_sqr] = kdtree_gridpcd.search_knn_vector_3d(p, k_nearest)
    for m in range(k):
        index.append(idx[m])

In [14]:
mask = np.ones(len(train_X), dtype=bool)
mask[index] = False
train_X = train_X[mask]
train_y = train_y[mask]

train_X = np.vstack([train_X, grasp_data])
train_y = np.hstack([train_y, fone])

In [15]:
train_X.shape

(1422, 3)

In [16]:
train_y

array([0., 0., 0., ..., 1., 1., 1.])

In [17]:
x_axis

array([-0.19272414, -0.14769434, -0.10266454, -0.05763474, -0.01260494,
        0.03242486,  0.07745466,  0.12248446,  0.16751425,  0.21254405])

In [18]:
xstar = np.zeros((res**3, 3))

for j in range(res):
	for i in range(res):
		d = j * res**2 # Delta
		axis_min = d + res * i
		axis_max = res * (i + 1) + d

		xstar[axis_min:axis_max, 0] = np.linspace(min_data[0], max_data[0], num=res) # in X
		xstar[axis_min:axis_max, 1] = min_data[1] + i * ((max_data[1] - min_data[1]) / res) # in X
		xstar[axis_min:axis_max, 2] = min_data[2] + ((j + 1) * ((max_data[2] - min_data[2]) / res))

tsize = res
xeva = np.reshape(xstar[:, 0], (tsize, tsize, tsize))
yeva = np.reshape(xstar[:, 1], (tsize, tsize, tsize))
zeva = np.reshape(xstar[:, 2], (tsize, tsize, tsize))

In [19]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        #self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
        # self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RQKernel())
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())
        # ThinPlateKernel
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [20]:
X = torch.FloatTensor(train_X).cuda()
y = torch.FloatTensor(train_y).cuda()

In [21]:
likelihood = gpytorch.likelihoods.GaussianLikelihood().cuda()
model =ExactGPModel(X,y,likelihood).cuda()
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

model.train()
likelihood.train()

GaussianLikelihood(
  (noise_covar): HomoskedasticNoise(
    (raw_noise_constraint): GreaterThan(1.000E-04)
  )
)

In [22]:
print("Predicting")
start = time.time()
optimizer = torch.optim.Adam([
    {'params': model.parameters()},  # Includes GaussianLikelihood parameters
], lr=0.1)


for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(X)
    # Calc loss and backprop gradients
    loss = -mll(output, y)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f   lengthscale: %.3f   noise: %.3f' % (
        i + 1, training_iter, loss.item(),
        model.covar_module.base_kernel.lengthscale.item(),
        model.likelihood.noise.item()
    ))
    optimizer.step()

Predicting
Iter 1/200 - Loss: 0.878   lengthscale: 0.693   noise: 0.693
Iter 2/200 - Loss: 0.846   lengthscale: 0.644   noise: 0.644
Iter 3/200 - Loss: 0.810   lengthscale: 0.598   noise: 0.598
Iter 4/200 - Loss: 0.768   lengthscale: 0.554   noise: 0.554
Iter 5/200 - Loss: 0.726   lengthscale: 0.513   noise: 0.513
Iter 6/200 - Loss: 0.687   lengthscale: 0.474   noise: 0.474
Iter 7/200 - Loss: 0.649   lengthscale: 0.437   noise: 0.437
Iter 8/200 - Loss: 0.613   lengthscale: 0.402   noise: 0.403
Iter 9/200 - Loss: 0.576   lengthscale: 0.371   noise: 0.370
Iter 10/200 - Loss: 0.543   lengthscale: 0.342   noise: 0.340
Iter 11/200 - Loss: 0.502   lengthscale: 0.316   noise: 0.312
Iter 12/200 - Loss: 0.468   lengthscale: 0.291   noise: 0.286
Iter 13/200 - Loss: 0.430   lengthscale: 0.268   noise: 0.262
Iter 14/200 - Loss: 0.387   lengthscale: 0.245   noise: 0.240
Iter 15/200 - Loss: 0.343   lengthscale: 0.224   noise: 0.219
Iter 16/200 - Loss: 0.303   lengthscale: 0.204   noise: 0.200
Iter 1

In [23]:
model.eval()
likelihood.eval()

GaussianLikelihood(
  (noise_covar): HomoskedasticNoise(
    (raw_noise_constraint): GreaterThan(1.000E-04)
  )
)

In [24]:
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    x = torch.FloatTensor(grasp_data).cuda()
    observed_pred = likelihood(model(x))
    original_prediction = observed_pred.mean.cpu().numpy()
    # print(pred.shape)
    original_confidence_lower, original_confidence_upper = observed_pred.confidence_region()

original_pred_low, original_pred_high = np.percentile(original_prediction, [display_percentile_low, display_percentile_high])

In [26]:
original_pred_low

0.9920201897621155

In [27]:
original_pred_high

1.0074458599090577

In [28]:
_ = time.time()
prediction_buf = []
uncertainty_buf = []
isplit=0
step_length = 10000
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    # test_x = torch.linspace(0, 1, 51)
    while isplit*step_length<xstar.shape[0]:
        isplit += 1
        split_min = step_length * (isplit-1)
        split_max = np.minimum(step_length * isplit, xstar.shape[0])

        xstar_tensor = torch.FloatTensor(xstar[split_min:split_max,:]).cuda()
        observed_pred = likelihood(model(xstar_tensor))
        prediction = observed_pred.mean.cpu().numpy()
        prediction_buf.append(prediction)
        confidence_lower, confidence_upper = observed_pred.confidence_region()
        confidence_lower = confidence_lower.cpu().numpy()
        confidence_upper = confidence_upper.cpu().numpy()
        uncertainty_buf.append(confidence_upper - confidence_lower)

In [29]:
prediction = np.hstack(prediction_buf)
uncertainty = np.hstack(uncertainty_buf)

In [30]:
prediction

array([-0.00996504, -0.01253034, -0.01430809, ..., -0.00672872,
       -0.00530795, -0.00312985], dtype=float32)

In [31]:
uncertainty

array([0.7974646 , 0.79237753, 0.7880502 , ..., 0.80020595, 0.8047056 ,
       0.8095431 ], dtype=float32)

In [ ]:
print("plotting")


fig = go.Figure(data=
            [go.Isosurface(
                x=xeva.flatten(),
                y=yeva.flatten(),
                z=zeva.flatten(),
                value=prediction.flatten(),
                isomin=original_pred_low,#-np.std(xtrain_prediction)/4 + np.mean(xtrain_prediction),
                isomax=original_pred_high,#np.std(xtrain_prediction)/4 + np.mean(xtrain_prediction), #right for bunny model
                caps=dict(x_show=False, y_show=False),
                # colorscale='RdBu',
                surface= dict(show=True,count=2, fill=0.6),

            ),
            go.Scatter3d(
                x=grasp_data[:,0],
                y=grasp_data[:,1],
                z=grasp_data[:,2],
                mode='markers',
                marker=dict(color="red")
            ),
            ]
    )
fig.show()